# Data Fetching with GitHub REST API
This notebook covers the way we fetched data from the [MLH Fellowship Organization](https://github.com/MLH-Fellowship) on GitHub to use for training in our hackathon project.

Table of Content:
- [List of Members](#first)
- [Total Commits](#second)
- [Followers and Open Repos](#third)
- [Stars and Forks on Repos](#fourth)
- [Number of Organizations](#fifth)
- [Number of Issues and Contributions](#sixth)

In [17]:
import requests # for API calls
import json # for JSON file type storage
import time # for API calls
import os # for the API token

token = os.getenv("GITHUB_TOKEN") # MUST HAVE PERSONAL ACCESS TOKEN SETUP
headers = {'Authorization': 'token {}'.format(token)}
base_url = "https://api.github.com"

## Get the members of the organization on GitHub and save them to a JSON file
### Member List <a class="anchor" id="first"></a>

In [2]:
# fetches all github users who are members of the MLH Fellowship organization on GitHub
def get_member_list(organization_name):
    members = []
    page_number = 1
    
    # iterate over pages of users, saving them to one pool
    while True:
        url = "{}/orgs/{}/members?per_page=100&page={}".format(base_url, organization_name, page_number)
        print("Calling URL: {}".format(url))
        response = requests.get(url, headers=headers)
        
        member_list = response.json()
        members += member_list
        
        print("There are {} in this URL call".format(len(member_list)))
        if len(member_list) == 0:
            break
        else:
            page_number += 1
            
    return members

In [3]:
# members = get_member_list("MLH-Fellowship")
with open('members.json', 'r') as json_file:
    members = json.load(json_file)

In [4]:
members[:1] # see the first user

[{'login': '1tracy',
  'id': 55264469,
  'node_id': 'MDQ6VXNlcjU1MjY0NDY5',
  'avatar_url': 'https://avatars.githubusercontent.com/u/55264469?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/1tracy',
  'html_url': 'https://github.com/1tracy',
  'followers_url': 'https://api.github.com/users/1tracy/followers',
  'following_url': 'https://api.github.com/users/1tracy/following{/other_user}',
  'gists_url': 'https://api.github.com/users/1tracy/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/1tracy/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/1tracy/subscriptions',
  'organizations_url': 'https://api.github.com/users/1tracy/orgs',
  'repos_url': 'https://api.github.com/users/1tracy/repos',
  'events_url': 'https://api.github.com/users/1tracy/events{/privacy}',
  'received_events_url': 'https://api.github.com/users/1tracy/received_events',
  'type': 'User',
  'site_admin': False}]

In [5]:
len(members) # there are 677 members in the organization

677

In [6]:
# save the list of members as a json file
with open('members.json', 'w') as json_file:
    json.dump(members, json_file)

## Get more specific user data for each user and save them to a JSON file
### Total Commits <a class="anchor" id="second"></a>

In [29]:
# Gets all of the commits the user commited
def get_total_commits(username):
    url = "{}/search/commits?q=author:{}".format(base_url, username)
    # requires a special header with an accept (search still in preview)
    headers = {"Content-Type": "application/json",
        "Accept": "application/vnd.github.cloak-preview",
        "Authorization": 'token {}'.format(token)}
    request = requests.get(url, headers=headers)
    return request.json()['total_count']

In [25]:
total = get_total_commits("dtemir")
print("I have a total of {} commits".format(total))

I have a total of 736 commits


### Followers and Open Repos <a class="anchor" id="third"></a>

In [26]:
# Gets the number of followers and repositories the user has
def get_followers_and_repos(username):
    url = "{}/users/{}".format(base_url, username)
    request = requests.get(url, headers=headers)
    return request.json()['followers'], request.json()['public_repos']

In [27]:
fl, re = get_followers_and_repos("dtemir")
print("I have {} followers and {} public repos".format(fl, re))

I have 17 followers and 13 public repos


### Stars and Forks on Repos <a class="anchor" id="fourth"></a>

In [46]:
# Gets the number of stars and forks the user has in their repositories
def get_stars_and_forks(username):
    url = "{}/users/{}/repos".format(base_url, username)
    request = requests.get(url, headers=headers)
    repositories = request.json()
    
    stars, forks = 0, 0
    for repository in repositories:
        stars += repository["stargazers_count"]
        forks += repository["forks_count"]
    
    return stars, forks

In [47]:
stars, forks = get_stars_and_forks("dtemir")
print("I have {} stars and {} forks on my repos".format(stars, forks))

### Number of Organizations  <a class="anchor" id="fifth"></a>

In [53]:
# Gets the number of organizations the user is a part of and their names
def get_organizations(username):
    url = "{}/users/{}/orgs".format(base_url, username)
    request = requests.get(url, headers=headers)
    organizations = request.json()
    
    number, names = 0, []
    for organization in organizations:
        number += 1
        names.append(organization["login"])
    
    return number, names

In [54]:
orgs, names = get_organizations("dtemir")
print("I'm a part of {} organizations named {}".format(orgs, names))

I'm a part of 2 organizations named ['ProteinDesignLab', 'MLH-Fellowship']


### Number of Issues and Contributions <a class="anchor" id="sixth"></a>

In [61]:
# Gets the number of Issues and PRs the user created in repositories
def get_issues_and_contributions(username):
    url_issues = "{}/search/issues?q=user:{}".format(base_url, username)
    url_contributions = "{}/search/issues?q=type:pr+user:{}".format(base_url, username)
    request_issues = requests.get(url_issues, headers=headers)
    request_contributions = requests.get(url_contributions, headers=headers)
    
    return request_issues.json()["total_count"], request_contributions.json()["total_count"]

In [62]:
issues, contributions = get_issues_and_prs("dtemir")
print("I have contributed {} issues and helped {} outside projects".format(issues, contributions))

I have contributed 8 issues and helped 6 outside projects
